<a href="https://colab.research.google.com/github/Juzz4vijay/Deep_Learning/blob/master/Finetune_Image_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install tensorflow==2.0 --upgrade
! pip install tensorflow-gpu==2.0 --upgrade
import tensorflow as tf
print(tf.__version__)

Requirement already up-to-date: tensorflow==2.0 in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: tensorflow-gpu==2.0 in /usr/local/lib/python3.6/dist-packages (2.0.0)
2.0.0


In [3]:
# example of loading the vgg16 model
from tensorflow.keras.applications.vgg16 import VGG16
# load model
model = VGG16()
# summarize the model
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
# example of loading the inception v3 model
from tensorflow.keras.applications.inception_v3 import InceptionV3
# load model
model = InceptionV3()
# summarize the model
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
# load model
model = ResNet50()
# summarize the model
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

# 1. MODEL_SELECTION

- INCEPTION_V3

- RESNET 50

- VGG 16

- DENSENET

In [0]:
# base_model = InceptionV3(weights='imagenet', include_top=False)
base_model = VGG16(weights='imagenet', include_top=False)

In [0]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout

In [0]:
CLASSES = 4
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
import pandas as pd
import numpy as np

In [14]:
test_path = "/content/drive/My Drive/Image_Autotag/dataset/Test Images/"
train_path = "/content/drive/My Drive/Image_Autotag/dataset/Train Images/"

train_df = pd.read_csv("/content/drive/My Drive/Image_Autotag/dataset/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Image_Autotag/dataset/test.csv")

from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
xtrain, xtest, ytrain, ytest = train_test_split(train_df['Image'], train_df['Class'], test_size=0.4, stratify=train_df['Class'], random_state=42)

In [0]:
xtrain.reset_index(drop=True, inplace=True)
ytrain.reset_index(drop=True, inplace=True)

In [17]:
image_corpus = []

image_class = []

for ind in xtrain.index:
  image_corpus.append(xtrain[ind])
  image_class.append(ytrain[ind])

  image_corpus.append(xtrain[ind].split(".jpg")[0] + '_GB.jpg')
  image_class.append(ytrain[ind])
  image_corpus.append(xtrain[ind].split(".jpg")[0] + '_SN.jpg')
  image_class.append(ytrain[ind])

df_train = pd.DataFrame()

df_train['Image'] = pd.Series(image_corpus)
df_train['Class'] = pd.Series(image_class)

df_train.head()

,Image,Class
0,image6713.jpg,Attire
1,image6713_GB.jpg,Attire
2,image6713_SN.jpg,Attire
3,image5639.jpg,Food
4,image5639_GB.jpg,Food


In [18]:
image_corpus = []

image_class = []

for ind in xtest.index:
  image_corpus.append(xtest[ind])
  image_class.append(ytest[ind])

  image_corpus.append(xtest[ind].split(".jpg")[0] + '_GB.jpg')
  image_class.append(ytest[ind])
  image_corpus.append(xtest[ind].split(".jpg")[0] + '_SN.jpg')
  image_class.append(ytest[ind])

df_valid = pd.DataFrame()

df_valid['Image'] = pd.Series(image_corpus)
df_valid['Class'] = pd.Series(image_class)
df_valid.head()

,Image,Class
0,image10266.jpg,misc
1,image10266_GB.jpg,misc
2,image10266_SN.jpg,misc
3,image2195.jpg,misc
4,image2195_GB.jpg,misc


In [19]:
print(len(df_train) + len(df_valid), (len(ytrain) + len(ytest)) * 3)

17949 17949


#Oversampling

In [20]:
df_train.Class.value_counts()

Food                    4101
Attire                  3042
misc                    2286
Decorationandsignage    1338
Name: Class, dtype: int64

In [0]:
ros = RandomOverSampler(random_state=42)

In [22]:
X_res, y_res = ros.fit_resample(df_train['Image'].values.reshape(-1,1), df_train['Class'].values.reshape(-1,1))

bal_df = pd.DataFrame()

bal_df["Image"] = pd.Series([item for sublist in X_res.tolist() for item in sublist])
bal_df["Class"] = pd.Series(y_res.tolist())

bal_df.Class.value_counts(), bal_df.Class.value_counts().sum()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(Attire                  4101
 misc                    4101
 Decorationandsignage    4101
 Food                    4101
 Name: Class, dtype: int64, 16404)

In [23]:
train_generator=data_generator.flow_from_dataframe(
dataframe=bal_df,
directory="/content/drive/My Drive/Image_Autotag/dataset/Train Images",
x_col="Image",
y_col="Class",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 16404 validated image filenames belonging to 4 classes.


In [24]:
valid_generator = data_generator.flow_from_directory(
    directory=r"/content/drive/My Drive/Image_Autotag/dataset/Valid Images",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 7182 images belonging to 4 classes.


In [25]:
len(ytest)*3

7182

In [26]:
%cd '/content/drive/My Drive/Image_Autotag'

/content/drive/My Drive/Image_Autotag


In [31]:
tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Image_Autotag', save_freq=10)

In [0]:
EPOCHS = 30
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS = 64
MODEL_FILE = 'VGG16_Image_Auto_Tag.h5'

history = model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_generator,
    validation_steps=VALIDATION_STEPS)

# serialize model to JSON
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Epoch 1/30
320/320 [==============================] - 5315s 17s/step - loss: 1.3616 - accuracy: 0.3319 - val_loss: 1.2798 - val_accuracy: 0.4004
Epoch 2/30
 78/320 [======>.......................] - ETA: 31:30 - loss: 1.2960 - accuracy: 0.3796

In [0]:
data = history.history

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

#Plot training and value accuracy values

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training and validation loss values

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

#INCEPTION V3:
Epoch 1/5
320/320 [==============================] - 3714s 12s/step - loss: 1.3537 - accuracy: 0.3877 - val_loss: 1.4939 - val_accuracy: 0.3931

Epoch 2/5
320/320 [==============================] - 1250s 4s/step - loss: 1.2598 - accuracy: 0.4496 - val_loss: 1.3328 - val_accuracy: 0.4609

Epoch 3/5
320/320 [==============================] - 224s 699ms/step - loss: 1.2368 - accuracy: 0.4612 - val_loss: 1.4148 - val_accuracy: 0.4150

Epoch 4/5
320/320 [==============================] - 220s 688ms/step - loss: 1.2082 - accuracy: 0.4777 - val_loss: 1.5767 - val_accuracy: 0.4033

Epoch 5/5
320/320 [==============================] - 221s 690ms/step - loss: 1.2237 - accuracy: 0.4745 - val_loss: 1.5996 - val_accuracy: 0.3652

#RESNET_50:

Epoch 1/5
320/320 [==============================] - 4621s 14s/step - loss: 1.3872 - accuracy: 0.4327 - val_loss: 1.4124 - val_accuracy: 0.2881

Epoch 2/5
320/320 [==============================] - 1650s 5s/step - loss: 1.2270 - accuracy: 0.4909 - val_loss: 1.6602 - val_accuracy: 0.2568

Epoch 3/5
320/320 [==============================] - 301s 939ms/step - loss: 1.1699 - accuracy: 0.5176 - val_loss: 1.6456 - val_accuracy: 0.3022

Epoch 4/5
320/320 [==============================] - 300s 938ms/step - loss: 1.1510 - accuracy: 0.5258 - val_loss: 1.5990 - val_accuracy: 0.2974

Epoch 5/5
320/320 [==============================] - 301s 939ms/step - loss: 1.1406 - accuracy: 0.5244 - val_loss: 1.7434 - val_accuracy: 0.2119

In [0]:
!cp -r '/content/VGG16_Image_Auto_Tag.model' '/content/drive/My Drive/Image_Autotag'